In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [5]:
df = pd.read_csv('cleaned_laptop_data1.csv')
#df.drop(columns=['Unnamed: 0','Gpu'],axis='columns',inplace=True)

In [6]:
df.head()

,Company,TypeName,Ram,Weight,Price,Touchscreen,IPS,ppi,Cpu brand,HDD,SSD,Gpu brand,os
0,Apple,Ultrabook,8,1.37,71378.6832,0,1,226.983005,Intel Core i5,0,128,Intel,Mac
1,Apple,Ultrabook,8,1.34,47895.5232,0,0,127.677940,Intel Core i5,0,0,Intel,Mac
2,HP,Notebook,8,1.86,30636.0000,0,0,141.211998,Intel Core i5,0,256,Intel,Others/No OS/Linux
3,Apple,Ultrabook,16,1.83,135195.3360,0,1,220.534624,Intel Core i7,0,512,AMD,Mac
4,Apple,Ultrabook,8,1.37,96095.8080,0,1,226.983005,Intel Core i5,0,256,Intel,Mac


In [11]:
new_col = {
    'Cpu brand':'Cpu_brand',
    'Gpu brand':'Gpu_brand'

}
df.rename(columns=new_col,inplace=True)

In [ ]:
Cpu brand

In [12]:
df.sample()

,Company,TypeName,Ram,Weight,Price,Touchscreen,IPS,ppi,Cpu_brand,HDD,SSD,Gpu_brand,os
1087,Lenovo,Notebook,8,1.7,63882.72,0,0,157.350512,Intel Core i5,0,180,Intel,Windows


In [13]:
df.to_csv("cleaned_laptop_data2.csv",index=False,header=True)

In [27]:
df.drop(columns=['ppi'],axis=1,inplace=True)

In [28]:
X = df.drop(columns=['Price'])
y = pd.DataFrame(np.log(df['Price']))

In [29]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.15,random_state=2)

In [30]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1106 entries, 183 to 1192
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Company      1106 non-null   object 
 1   TypeName     1106 non-null   object 
 2   Ram          1106 non-null   int64  
 3   Weight       1106 non-null   float64
 4   Touchscreen  1106 non-null   int64  
 5   IPS          1106 non-null   int64  
 6   Cpu brand    1106 non-null   object 
 7   HDD          1106 non-null   int64  
 8   SSD          1106 non-null   int64  
 9   Gpu brand    1106 non-null   object 
 10  os           1106 non-null   object 
dtypes: float64(1), int64(5), object(5)
memory usage: 103.7+ KB


In [31]:
step1 = ColumnTransformer(transformers=[
    ('col_tnf',OneHotEncoder(drop='first',sparse_output=False),[0,1,6,9,10])
],remainder='passthrough')

step2 = RandomForestRegressor(n_estimators=100,
                            random_state=3,
                            max_samples=0.5,
                            max_features=0.75,
                            max_depth=15)

pipe = Pipeline([
    ('step1',step1),
    #('step2',step2)
])

X_train_encoded = step1.fit_transform(X_train)
X_test_encoded = step1.transform(X_test)

step2.fit(X_train_encoded,y_train)

y_pred = step2.predict(X_test_encoded)

print('R2 score',r2_score(y_test,y_pred))
print('MAE',mean_absolute_error(y_test,y_pred))

d:\propredict\pp\lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


R2 score 0.8879698349726288
MAE 0.15756874019613865


In [32]:

import pickle


In [33]:
pickle.dump(step1,open('pipe.pkl','wb'))